# HDR
HDR lo que hace es tomar fotos a diferente grado de exposición para poder tener un rango de colores más fiel al real. Cuando uno saca una foto hay zonas el brillo se ajusta a un punto de foco generando saturaciones y colores posiblemente no fieles. HDR toma fotos a diferentes grados de exposición, las ajusta para que se alinien los objetos en ellas y luego las combina para formar una imagen con un rango de color más cercano al real.

In [1]:
import numpy as np
import cv2

In [2]:
ASSETS_FOLDER = "./assets/HDR_images"

In [3]:
def readImagesAndTimes():
  # List of exposure times
  times = np.array([ 0.0333, 0.1000, 0.3333, 0.6250, 1.3000, 4.0000], dtype=np.float32)
 
  # List of image filenames
  filenames = [f"{ASSETS_FOLDER}/office_1.jpg", 
               f"{ASSETS_FOLDER}/office_2.jpg", 
               f"{ASSETS_FOLDER}/office_3.jpg", 
               f"{ASSETS_FOLDER}/office_4.jpg", 
               f"{ASSETS_FOLDER}/office_5.jpg", 
               f"{ASSETS_FOLDER}/office_6.jpg"]
  images = []
  for filename in filenames:
    im = cv2.imread(filename)
    images.append(im)
 
  return images, times

In [4]:
images, times = readImagesAndTimes()

In [5]:
alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

In [6]:
calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)

In [7]:
# Merge images into an HDR linear image
mergeDebevec = cv2.createMergeDebevec()
hdrDebevec = mergeDebevec.process(images, times, responseDebevec)
# Save HDR image.
cv2.imwrite("hdrDebevec.hdr", hdrDebevec)

True

In [8]:
tonemapDrago = cv2.createTonemapDrago(1.0, 0.7)
ldrDrago = tonemapDrago.process(hdrDebevec)
ldrDrago = 3 * ldrDrago
cv2.imwrite("ldr-Drago.jpg", ldrDrago * 255)

[ WARN:0@0.333] global matrix_expressions.cpp:1333 assign OpenCV/MatExpr: processing of multi-channel arrays might be changed in the future: https://github.com/opencv/opencv/issues/16739


True

In [9]:
# Tonemap using Reinhard's method to obtain 24-bit color image
tonemapReinhard = cv2.createTonemapReinhard(1.5, 0,0,0)
ldrReinhard = tonemapReinhard.process(hdrDebevec)
cv2.imwrite("ldr-Reinhard.jpg", ldrReinhard * 255)

True

In [10]:
# Tonemap using Mantiuk's method to obtain 24-bit color image
tonemapMantiuk = cv2.createTonemapMantiuk(2.2,0.85, 1.2)
ldrMantiuk = tonemapMantiuk.process(hdrDebevec)
ldrMantiuk = 3 * ldrMantiuk
cv2.imwrite("ldr-Mantiuk.jpg", ldrMantiuk * 255)

True